In [4]:
from google.colab import drive 
drive.mount('/content/drive') 

Mounted at /content/drive


In [5]:
import sqlite3
import pandas as pd

In [6]:
# In order to load the data.
con= sqlite3.connect('/content/drive/My Drive/Classroom/dataset/Db-IMDB.db')

In [7]:
table = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'",con)
table 

,type,name,tbl_name,rootpage,sql
0,table,Movie,Movie,2,"CREATE TABLE ""Movie"" (\n""index"" INTEGER,\n ""M..."
1,table,Genre,Genre,4,"CREATE TABLE ""Genre"" (\n""index"" INTEGER,\n ""N..."
2,table,Language,Language,5,"CREATE TABLE ""Language"" (\n""index"" INTEGER,\n ..."
3,table,Country,Country,6,"CREATE TABLE ""Country"" (\n""index"" INTEGER,\n ..."
4,table,Location,Location,7,"CREATE TABLE ""Location"" (\n""index"" INTEGER,\n ..."
5,table,M_Location,M_Location,11,"CREATE TABLE ""M_Location"" (\n""index"" INTEGER,\..."
6,table,M_Country,M_Country,10,"CREATE TABLE ""M_Country"" (\n""index"" INTEGER,\n..."
7,table,M_Language,M_Language,9,"CREATE TABLE ""M_Language"" (\n""index"" INTEGER,\..."
8,table,M_Genre,M_Genre,8,"CREATE TABLE ""M_Genre"" (\n""index"" INTEGER,\n ..."
9,table,Person,Person,12,"CREATE TABLE ""Person"" (\n""index"" INTEGER,\n ""..."


![alt text](https://drive.google.com/uc?id=1B_Ymu7AQRaBGDQiD6ww6QULkUS7WdfTA)

**1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check  that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.**

In [ ]:
pd.read_sql_query("SELECT m.title, cast(m.year AS INTEGER) as cyear, p.Name FROM Movie m, Person p, Genre g INNER JOIN M_Director md ON md.MID = m.MID AND md.PID = p.PID INNER JOIN M_Genre mg ON mg.MID = m.MID AND mg.GID = g.GID  WHERE g.Name LIKE '%Comedy%' AND ((cyear % 4 = 0 AND cyear % 100 !=0) OR cyear % 400 =0) GROUP BY m.title", con)

,title,cyear,Name
0,100% Love,2012,Rabi Kinagi
1,3 Bachelors,2012,Ajai Sinha
2,99,0,Krishna D.K.
3,A Flying Jatt,2016,Remo D'Souza
4,Ab Ayega Mazaa,1984,Pankaj Parashar
...,...,...,...
240,Welcome,0,Anees Bazmee
241,Welcome to Sajjanpur,2008,Shyam Benegal
242,Will You Marry Me,2012,Aditya Datt
243,Yaad Rakhegi Duniya,1992,Deepak Anand


**2. List the names of all the actors who played in the movie 'Anand' (1971)**


In [ ]:
#why this code is not worked for me
pd.read_sql_query("SELECT Name FROM Person WHERE PID IN (SELECT PID FROM M_Cast WHERE MID IN (SELECT MID FROM Movie where title='Anand'))",con)

,Name


In [ ]:
pd.read_sql_query("SELECT p.Name FROM Movie m JOIN M_Cast mc ON m.MID=mc.MID JOIN Person p ON trim(mc.PID)=trim(p.PID) WHERE m.title='Anand' AND m.year=1971", con)

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


**3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)**

In [ ]:
#why this code is not worked for me //////? 
pd.read_sql_query("SELECT Name FROM Person WHERE PID IN ((SELECT PID FROM M_Cast WHERE MID IN (SELECT MID FROM movie WHERE year<1970)) AND (SELECT PID FROM M_Cast WHERE MID IN (SELECT MID FROM movie WHERE year> 1990)))",con)

,Name


In [ ]:
pd.read_sql_query("SELECT DISTINCT trim(p.PID) as pid, p.Name FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID JOIN Person p ON trim(mc.PID) = trim(p.PID) WHERE m.year > 1990 AND trim(p.PID) IN (SELECT DISTINCT trim(p.PID) as pid FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID JOIN Person p ON trim(mc.PID) = trim(p.PID) WHERE m.year < 1970) GROUP BY trim(p.PID)",con)

,pid,Name
0,nm0000821,Amitabh Bachchan
1,nm0003987,Mohandas K. Gandhi
2,nm0004292,Raj Kapoor
3,nm0004334,Rekha
4,nm0004429,Dharmendra
...,...,...
328,nm2983292,Mohamad Ali
329,nm4563339,Dube
330,nm7390393,Aachi Manorama
331,nm9036653,Surendra Rahi


**4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.** 

In [ ]:
pd.read_sql_query("SELECT p.Name, count(md.ID) movieCount FROM M_Director md JOIN Person p ON md.PID=p.PID GROUP BY md.PID HAVING movieCount >= 10 ORDER BY movieCount DESC",con)

,Name,movieCount
0,David Dhawan,78
1,Mahesh Bhatt,70
2,Ram Gopal Varma,60
3,Vikram Bhatt,58
4,Hrishikesh Mukherjee,54
...,...,...
151,Siddharth Anand,10
152,Dibakar Banerjee,10
153,Shoojit Sircar,10
154,R. Balki,10


**5.** 
- **a.** For each year, count the number of movies in that year that had only female actors.
- **b.** Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.**

In [ ]:
pd.read_sql_query("SELECT count(m.year) movie_count, m.year as movie_year FROM Movie m where m.MID not in (SELECT mc.MID FROM Person p JOIN M_Cast mc ON trim(p.PID)=trim(mc.PID) WHERE p.Gender='Male') GROUP BY movie_year ORDER BY movie_count DESC", con)

,movie_count,movie_year
0,1,1939
1,1,1999
2,1,2000
3,1,2009
4,1,2012
5,1,2018
6,1,I 2018


In [ ]:
pd.read_sql_query(
            '''
            SELECT y.allMov as 'movie_count', x.year as movie_year, ((x.Movies_Cnt*100.0)/y.allMov) as Percent FROM
            (SELECT count(*) Movies_Cnt , m.year
            FROM Movie m where m.MID not in
            (SELECT mc.MID FROM Person p JOIN M_Cast mc ON trim(p.PID) = trim(mc.PID) WHERE p.Gender='Male')
            GROUP BY m.year) x INNER JOIN
            (SELECT count(*) allMov, m.year
            FROM Movie m
            GROUP BY m.year) y on x.year=y.year
            ''', con)

,movie_count,movie_year,Percent
0,2,1939,50.000000
1,66,1999,1.515152
2,64,2000,1.562500
3,100,2009,1.000000
4,109,2012,0.917431
5,93,2018,1.075269
6,10,I 2018,10.000000


**6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.**

In [ ]:
pd.read_sql_query(
            '''
            SELECT count(DISTINCT mc.PId) as cast_count, m.title FROM Movie m
            JOIN M_Cast mc ON m.MID=mc.MID
            JOIN Person p ON trim(mc.PID)=trim(p.PID)
            GROUP BY m.MID
            ORDER BY cast_count DESC limit 1
            ''', con)

,cast_count,title
0,238,Ocean's Eight


**7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.**

In [ ]:
pd.read_sql_query('''
            SELECT d_year.year AS start, d_year.year+9 AS end, count(1) AS total_movies FROM
            (SELECT DISTINCT(year) FROM Movie) d_year
            JOIN Movie m WHERE m.year>=start AND m.year<=end
            GROUP BY end
            ORDER BY total_movies DESC
            LIMIT 1
            ''', con)

,start,end,total_movies
0,2008,2017,1128


**8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).**

In [ ]:
pd.read_sql_query("select p1.Name from Person p1 where p1.pid not in (select distinct(mc1.pid) from M_Cast as mc1 join Movie as m1 on mc1.mid=m1.mid where\
      exists(select mc2.mid from M_Cast as mc2 join Movie as m2 on mc2.mid=m2.mid where trim(mc1.PID) = trim(mc2.PID) and (substr(m2.year,-4,4) - 3) > substr(m1.year,-4,4) and\
      not exists(select trim(mc3.MID) from M_Cast as mc3 join Movie as m3 on mc3.mid=m3.mid where trim(mc1.PID) = trim(mc3.PID) and substr(m1.year,-4,4) < substr(m3.year,-4,4) and\
      substr(m3.year,-4,4) < substr(m2.year,-4,4))));",con)

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
...,...
38280,Kannan
38281,Adrian Fulle
38282,Gulshan Kumar
38283,Iqbal


**9.Find all the actors that made more movies with Yash Chopra than any other director.**

In [ ]:
pd.read_sql_query("SELECT p.NAME,max_movies FROM   person p INNER JOIN (SELECT director_pid,Max(movies) AS max_movies,actor_pid FROM (SELECT Count(mc.mid) AS Movies,md.pid AS Director_PID, mc.pid AS Actor_PID FROM m_director md INNER JOIN (SELECT * FROM m_cast WHERE Trim(pid) IN (SELECT Trim(pid) FROM m_cast WHERE Trim(mid) IN (SELECT Trim(mid)FROM m_director WHERE Trim(pid) = (SELECT Trim(pid)FROM person WHERE  NAME LIKE '%Yash Chopra%'))))as mc ON Trim(md.mid) = Trim(mc.mid)GROUP  BY actor_pid, director_pid)GROUP  BY actor_pid)ON Trim(p.pid) = Trim(actor_pid) WHERE  director_pid = (SELECT Trim(pid) FROM person WHERE  NAME LIKE '%Yash Chopra%')AND Trim(p.pid) <> (SELECT Trim(pid)FROM   person WHERE  NAME LIKE '%Yash Chopra%') ORDER  BY max_movies DESC", con)

,Name,max_movies
0,Jagdish Raj,11
1,Manmohan Krishna,10
2,Manmohan Krishna,10
3,Iftekhar,9
4,Shashi Kapoor,7
...,...,...
226,Naval,1
227,Prem Sood,1
228,Ramlal Shyamlal,1
229,Aziz Mirza,1


**10 The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.**

In [ ]:
pd.read_sql_query('''SELECT DISTINCT P.Name
            FROM Person p
                     JOIN M_Cast mc
                          ON trim(p.PID) = trim(mc.PID)
            WHERE mc.MID IN (SELECT mc.MID
                             FROM M_Cast mc
                             WHERE trim(mc.PID) IN (
                                 SELECT trim(p.PID) as pid
                                 FROM Person p
                                          JOIN M_Cast mc
                                               ON trim(p.PID) = trim(mc.PID)
                                 WHERE mc.MID IN (
                                     SELECT mc.MID
                                     FROM Person p
                                              JOIN M_Cast mc
                                                   ON trim(p.PID) = trim(mc.PID)
                                     WHERE trim(p.Name) LIKE '%shah rukh khan%'
                                 )
                                   AND trim(p.Name) NOT LIKE '%shah rukh khan%'
                             )
                               AND mc.MID NOT IN (SELECT mc.MID
                                                  FROM Person p
                                                           JOIN M_Cast mc
                                                                ON trim(p.PID) = trim(mc.PID)
                                                  WHERE trim(p.Name) LIKE '%shah rukh khan%'))
            
            ''', con)

,Name
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
...,...
25964,Minoo Mehtab
25965,Hayley Cleghorn
25966,Nirvasha Jithoo
25967,Kamal Maharshi
